In [1]:
import pandas as pd
import requests

In [2]:
api_key = 'HM57HIQECDPJW6E7'

In [3]:
symbol = 'AAPL'

In [4]:
base_url = 'https://www.alphavantage.co/query'

In [17]:
url = f"{base_url}?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}"

In [18]:
r = requests.get(url)

In [28]:
r

<Response [200]>

In [29]:
data = r.json()

In [30]:
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-08-16',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-08-16': {'1. open': '177.1300',
   '2. high': '178.5400',
   '3. low': '176.5000',
   '4. close': '176.5700',
   '5. volume': '46964857'},
  '2023-08-15': {'1. open': '178.8800',
   '2. high': '179.4800',
   '3. low': '177.0500',
   '4. close': '177.4500',
   '5. volume': '43622593'},
  '2023-08-14': {'1. open': '177.9700',
   '2. high': '179.6900',
   '3. low': '177.3050',
   '4. close': '179.4600',
   '5. volume': '43675627'},
  '2023-08-11': {'1. open': '177.3200',
   '2. high': '178.6200',
   '3. low': '176.5500',
   '4. close': '177.7900',
   '5. volume': '52036672'},
  '2023-08-10': {'1. open': '179.4800',
   '2. high': '180.7500',
   '3. low': '177.6000',
   '4. close': '177.9700',
   '5. volume': '54686851'},
  '2023-08-09': {'1. open': '180

In [31]:
type(data)

dict

In [32]:
#convert data to DataFrame
df = pd.DataFrame(data)

In [33]:
df

,Meta Data,Time Series (Daily)
1. Information,"Daily Prices (open, high, low, close) and Volumes",NaN
2. Symbol,AAPL,NaN
3. Last Refreshed,2023-08-16,NaN
4. Output Size,Compact,NaN
5. Time Zone,US/Eastern,NaN
...,...,...
2023-03-30,NaN,"{'1. open': '161.5300', '2. high': '162.4700',..."
2023-03-29,NaN,"{'1. open': '159.3700', '2. high': '161.0500',..."
2023-03-28,NaN,"{'1. open': '157.9700', '2. high': '158.4900',..."
2023-03-27,NaN,"{'1. open': '159.9400', '2. high': '160.7700',..."


In [46]:
#downloaded csv file of AAPL stockprices from Neptune.ai

In [47]:
stockprices = pd.read_csv("AAPL_Stock_Data.csv", index_col="Date")

In [49]:
stockprices

,Low,High,Close,Open
Date,,,,
1999-11-01,77.370,80.69,77.62,80.00
1999-11-02,77.310,81.69,80.25,78.00
1999-11-03,81.000,83.25,81.50,81.62
1999-11-04,80.620,85.37,83.62,82.06
1999-11-05,84.000,88.37,88.31,84.62
...,...,...,...,...
2021-06-29,134.350,136.49,136.33,134.80
2021-06-30,135.870,137.41,136.96,136.17
2021-07-01,135.760,137.33,137.27,136.60


In [55]:
#80-20 train-test split
test_ratio = 0.2
training_ratio = 1 - test_ratio

In [69]:
train_size = int(training_ratio * len(stockprices))
test_size = int(test_ratio * len(stockprices))
print(f'train_size: {train_size}')

train_size: 4363


In [70]:
print(f'test_size: {test_size}')

test_size: 1090


In [71]:
#training with first 80%
train = stockprices[:train_size][['Close']]

In [72]:
#testing with last 20%
test = stockprices[train_size:][['Close']]

In [73]:
## Split the time-series data into training seq X and output value Y
def extract_seqX_outcomeY(data, N, offset):
    """
    Split time-series into training sequence X and outcome value Y
    Args:
        data - dataset
        N - window size, e.g., 50 for 50 days of historical stock prices
        offset - position to start the split
    """
    X = []
    y = []

    for i in range(offset, len(data)):
        X.append(data[i - N : i])
        y.append(data[i])

    return np.array(X), np.array(y)